<a href="https://colab.research.google.com/github/hydrationn/Machine-DeepLearning/blob/main/DeepLearning/lab/20240418_%EC%8B%A4%EC%8A%B503_Regularization_and_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Practice 1. Dropout

In [ ]:
# import package

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.utils.data as data_utils
import torchvision
import torchvision.transforms as transforms

import numpy as np
import matplotlib.pyplot as plt
import random

In [ ]:
# GPU/CPU setting

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
  torch.cuda.manual_seed_all(777)

cpu


In [ ]:
# import dataset

train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor(), download=True)

100%|██████████| 9912422/9912422 [00:00<00:00, 95853772.75it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 84356332.75it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 30045448.57it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 5772887.51it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [ ]:
# Batch generation

batch_size = 100

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# Model

class MLP_Dropout(nn.Module):
  def __init__(self):
    super(MLP_Dropout, self).__init__()
    self.fc1 = nn.Linear(784, 256)
    self.fc2 = nn.Linear(256, 64)
    self.fc3 = nn.Linear(64, 10)
    self.dp1 = nn.Dropout(p=0.4)
    self.dp2 = nn.Dropout(p = 0.2)

  def forward(self, x):
    h1 = F.relu(self.fc1(x))
    h1dp = self.dp1(h1)
    h2 = F.relu(self.fc2(h1dp))
    h2dp = self.dp2(h2)
    output = self.fc3(h2dp)
    return output

In [ ]:
# Parameter, model, loss function, optimizer

# 파라미터, 모델 정의
num_epochs = 10
learning_rate = 0.01

model = MLP_Dropout().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# Train

# Model Train
loss_list = []
for epoch_num in range(num_epochs):
  average_cost = 0
  model.train()

  for batch_idx, (x_data, y_label) in enumerate(train_loader):
    num_of_mini_batch = len(train_loader)

    x_data = x_data.reshape(-1, 28*28)
    input_image = x_data.to(device)
    label = y_label.to(device)
    optimizer.zero_grad()
    y_predict = model(input_image)
    loss = criterion(y_predict, label)
    loss.backward()
    optimizer.step()

    average_cost = average_cost + (loss.item()/num_of_mini_batch)
    loss_list.append(loss)

  print("Epoch {} Loss {:.5f}".format((epoch_num+1), average_cost))

Epoch 1 Loss 0.36812
Epoch 2 Loss 0.27414
Epoch 3 Loss 0.24278
Epoch 4 Loss 0.23866
Epoch 5 Loss 0.22997
Epoch 6 Loss 0.21457
Epoch 7 Loss 0.20419
Epoch 8 Loss 0.21019
Epoch 9 Loss 0.20264
Epoch 10 Loss 0.21395


In [ ]:
# Validation

with torch.no_grad():
  num_total_data = 0
  correct = 0
  model.eval()
  for batch_idx, (images, labels) in enumerate(test_loader):

    images = images.reshape(-1, 28*28)
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    outputs_softmax = F.softmax(outputs, dim=1)
    predicted = torch.argmax(outputs_softmax, dim=1)

    num_total_data += len(images)
    answer = sum(labels==predicted).item()
    correct += answer

print("Model accuracy {:.5f}%".format((correct/num_total_data)*100))

Model accuracy 96.72000%


In [ ]:
# Practice 2. Batch nomalization

In [ ]:
# import package

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.utils.data as data_utils
import torchvision
import torchvision.transforms as transforms

import numpy as np
import matplotlib.pyplot as plt
import random

In [ ]:
# GPU/CPU setting

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
  torch.cuda.manual_seed_all(777)

cpu


In [ ]:
# import dataset

train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor(), download=True)

In [ ]:
# Batch generation

batch_size = 100

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# Model

class Batch_Norm(nn.Module):
  def __init__(self):
    super(Batch_Norm, self).__init__()

    self.network = nn.Sequential(nn.Linear(784, 256),
                                 nn.BatchNorm1d(256),
                                 nn.ReLU(),
                                 nn.Linear(256, 64),
                                 nn.BatchNorm1d(64),
                                 nn.ReLU(),
                                 nn.Linear(64, 10))

  def forward(self, x):
    output = self.network(x)

    return output

In [ ]:
# Hyperparameters

num_epochs = 10
learning_rate = 0.01

model = Batch_Norm().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# Train

# Model Train
loss_list = []
for epoch_num in range(num_epochs):
  average_cost = 0
  model.train()

  for batch_idx, (x_data, y_label) in enumerate(train_loader):
    num_of_mini_batch = len(train_loader)

    x_data = x_data.reshape(-1, 28*28)
    input_image = x_data.to(device)
    label = y_label.to(device)
    optimizer.zero_grad()
    y_predict = model(input_image)
    loss = criterion(y_predict, label)
    loss.backward()
    optimizer.step()

    average_cost = average_cost + (loss.item()/num_of_mini_batch)
    loss_list.append(loss)

  print("Epoch {} Loss {:.5f}".format((epoch_num+1), average_cost))

Epoch 1 Loss 0.02289
Epoch 2 Loss 0.02098
Epoch 3 Loss 0.01763
Epoch 4 Loss 0.02005
Epoch 5 Loss 0.01910
Epoch 6 Loss 0.01423
Epoch 7 Loss 0.01579
Epoch 8 Loss 0.01666
Epoch 9 Loss 0.01383
Epoch 10 Loss 0.01254


In [ ]:
# Validation

with torch.no_grad():
  num_total_data = 0
  correct = 0
  model.eval()
  for batch_idx, (images, labels) in enumerate(test_loader):

    images = images.reshape(-1, 28*28)
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    outputs_softmax = F.softmax(outputs, dim=1)
    predicted = torch.argmax(outputs_softmax, dim=1)

    num_total_data += len(images)
    answer = sum(labels==predicted).item()
    correct += answer

print("Model accuracy {:.5f}%".format((correct/num_total_data)*100))

Model accuracy 97.91000%
